In [1]:
import sys
import os
import pandas as pd

# Add project root to Python path to find the 'src' directory
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added project root to sys.path: {project_root}")

# Import your new ConvLSTM pipeline class
from src.improved_convlstm_multitask_pipeline import MultitaskConvLSTMPipeline

Added project root to sys.path: c:\Users\peera\Desktop\DroughtLSTM_oneday


c:\Users\peera\.conda\envs\drought_lstm_base\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch, PyTorch Lightning, and Optuna successfully imported.
ConvLSTM Pipeline: Successfully imported utility functions.


c:\Users\peera\.conda\envs\drought_lstm_base\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.1 is exactly one major version older than the runtime version 6.31.1 at api.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


In [2]:
%pwd

'c:\\Users\\peera\\Desktop\\DroughtLSTM_oneday\\notebooks'

In [3]:
# --- 1. Setup ---
import os
from src.cam_clstm.cam_convlstm_pipeline import CAMConvLSTMPipeline
config_path = "../config/camclstm/cam_config.yaml"

In [4]:
from src.cam_clstm.convert_csv_to_lstm_dataset import convert_csv_to_lstm_dataset
from torch.utils.data import DataLoader
from src.cam_clstm.causual_clstm_pipeline import GriddedConvLSTMDatasetWithMeta,GriddedConvLSTMDataset
from src.data_utils import load_and_prepare_data,load_config


In [ ]:

config = load_config(config_path)
df = load_and_prepare_data(config)
df = df[col for col in df.columns if col.startswith('cam_') or col.startswith('meteo_') or col.startswith('target_')]]


Configuration loaded from ../config/camclstm/cam_config.yaml
Successfully loaded data from ../data/processed/full_scaled.csv. Shape: (264204, 19)
Converted column 'time' to datetime.
Data sorted by ['time', 'lat', 'lon'].


In [6]:
from src.data_utils import split_data_chronologically
df_train, df_val, df_test = split_data_chronologically(df, config)

X_train, y_train_dict, _ = convert_csv_to_lstm_dataset(df_train, config, seq_len=12)
X_val,   y_val_dict, _ = convert_csv_to_lstm_dataset(df_val, config, seq_len=12)
X_test,  y_test_dict, _ = convert_csv_to_lstm_dataset(df_test, config, seq_len=12)



Splitting data: Train ends 2017-12-31 00:00:00, Validation ends 2020-12-31 00:00:00
Train set shape: (251316, 19), Time range: 1901-01-16 00:00:00 to 2017-12-16 00:00:00
Validation set shape: (6444, 19), Time range: 2018-01-16 00:00:00 to 2020-12-16 00:00:00
Test set shape: (6444, 19), Time range: 2021-01-16 00:00:00 to 2023-12-16 00:00:00
--- Starting Data Gridding Process (Fixed Step Method) ---
Using fixed grid step of: 0.5 degrees
Grid boundaries: LAT (6.25, 20.25), LON (97.75, 105.25)
Calculated grid dimensions: Height=29, Width=16
Created 2D validity mask (29x16) with 179 valid data pixels.
Pivoting data into a 4D tensor of shape (1404, 29, 16, 15)...


c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row_idx'] = ((df[lat_col] - lat_min) / fixed_step).round().astype(int)
c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['col_idx'] = ((df[lon_col] - lon_min) / fixed_step).round().astype(int)


--- Data Gridding Process Finished ---
Created sequence data: X=(1392, 12, 14, 29, 16), y=(1392, 1, 29, 16)
--- Starting Data Gridding Process (Fixed Step Method) ---
Using fixed grid step of: 0.5 degrees
Grid boundaries: LAT (6.25, 20.25), LON (97.75, 105.25)
Calculated grid dimensions: Height=29, Width=16
Created 2D validity mask (29x16) with 179 valid data pixels.
Pivoting data into a 4D tensor of shape (36, 29, 16, 15)...


c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row_idx'] = ((df[lat_col] - lat_min) / fixed_step).round().astype(int)
c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['col_idx'] = ((df[lon_col] - lon_min) / fixed_step).round().astype(int)
c:\Users\peera\Desktop\DroughtLSTM_oneday\src\grid_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

--- Data Gridding Process Finished ---
Created sequence data: X=(24, 12, 14, 29, 16), y=(24, 1, 29, 16)
--- Starting Data Gridding Process (Fixed Step Method) ---
Using fixed grid step of: 0.5 degrees
Grid boundaries: LAT (6.25, 20.25), LON (97.75, 105.25)
Calculated grid dimensions: Height=29, Width=16
Created 2D validity mask (29x16) with 179 valid data pixels.
Pivoting data into a 4D tensor of shape (36, 29, 16, 15)...
--- Data Gridding Process Finished ---
Created sequence data: X=(24, 12, 14, 29, 16), y=(24, 1, 29, 16)


In [7]:
#get lat,lon,time
lat = df_train['lat'].unique()
lon = df_train['lon'].unique()
#get time for train,val,test
time_train = df_train['time'].drop_duplicates().sort_values().values
time_val = df_val['time'].drop_duplicates().sort_values().values
time_test = df_test['time'].drop_duplicates().sort_values().values

In [8]:
from torch.utils.data import DataLoader
import torch
def collate_drop_meta(batch):
    """Removes metadata so Lightning won't fail"""
    xs, ys, _ = zip(*batch)
    return torch.utils.data.default_collate(xs), torch.utils.data.default_collate(ys)

#train_dataset = GriddedConvLSTMDatasetWithMeta(X_train, y_train_dict,time_train, lat, lon)
#val_dataset   = GriddedConvLSTMDatasetWithMeta(X_val, y_val_dict, time_val, lat, lon)
#test_dataset  = GriddedConvLSTMDatasetWithMeta(X_test, y_test_dict, time_test, lat, lon)

#train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True,collate_fn=collate_drop_meta)
#val_loader   = DataLoader(val_dataset, batch_size=8,shuffle=False, collate_fn=collate_drop_meta)
#test_loader  = DataLoader(test_dataset, batch_size=8,shuffle=False, collate_fn=collate_drop_meta)
train_dataset = GriddedConvLSTMDataset(X_train, y_train_dict)
val_dataset   = GriddedConvLSTMDataset(X_val, y_val_dict)
test_dataset  = GriddedConvLSTMDataset(X_test, y_test_dict)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [9]:
from src.cam_clstm.causal_clsm_model import MyConvLSTMModel
from src.cam_clstm.causal_multitask_lightning_module import CausalMultitaskLightningModule
from pytorch_lightning import Trainer
train_dataset
model = MyConvLSTMModel(
    input_channels=X_train.shape[2],
    height=X_train.shape[3],
    width=X_train.shape[4],
    hidden_channels=32,
    use_pos_enc=True,
    use_spatial_attn=True,
    use_temporal_only=True
)

lightning_module = CausalMultitaskLightningModule(model, config)
trainer = Trainer(max_epochs=50)
trainer.fit(lightning_module, train_loader, val_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type            | Params | Mode 
----------------------------------------------------
0 | model   | MyConvLSTMModel | 62.0 K | train
1 | loss_fn | MSELoss         | 0      | train
----------------------------------------------------
62.0 K    Trainable params
0         Non-trainable params
62.0 K    Total params
0.248     Total estimated model params size (MB)
33        Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\peera\AppData\Roaming\Python\Python312\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


RuntimeError: Given groups=1, weight of size [8, 18, 3, 3], expected input[96, 14, 29, 16] to have 18 channels, but got 14 channels instead